In [1]:
import json
import os

port_start = 60066
tf_config = {
    'cluster': {
        'worker': [f'ferrari:{port_start}', f'volvo:{port_start+1}']
    },
    #'task': {'type': 'worker', 'index': 1} #For volvo
    'task': {'type': 'worker', 'index': 0}
}

json.dumps(tf_config)
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [2]:
import json
import os
import tqdm

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import lib_stylegan

tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

im_size = 128
latent_size = 256 
channels = 24 # Should be at least 32 for good results
#Chosing the number of layer this way, means we start with 4x4
nb_layer = int(np.log2(im_size) - 1) 

path="/Data/leo/dogs-face-2015/*jpg"

per_worker_batch_size = 16
global_batch_size = per_worker_batch_size * num_workers


strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
global_batch_size = per_worker_batch_size * num_workers

with strategy.scope():
    model = lib_stylegan.style_gan.StyleGan(im_size=im_size, 
                                            latent_size=latent_size, 
                                            channels=channels,
                                            nb_layer=nb_layer)
                                            global_batch_size=global_batch_size)
    model.compile(run_eagerly=True)

dataset = lib_stylegan.dataset.train_dataset(path, 
                                             n_layers=nb_layer, 
                                             im_size=im_size, 
                                             batch_size=per_worker_batch_size,
                                             latent_size=latent_size
                                            )
print("Here")
for args in tqdm.tqdm(dataset.take(100), total=100):
    per_replica_losses = strategy.run(model.train_step, args=(args,))
for args in tqdm.tqdm(dataset.take(1), total=1):
    per_replica_losses = strategy.run(model.train_step, args=(args,))

INFO:tensorflow:Enabled multi-worker collective ops with available devices: ['/job:worker/replica:0/task:0/device:CPU:0', '/job:worker/replica:0/task:0/device:XLA_CPU:0', '/job:worker/replica:0/task:0/device:XLA_GPU:0', '/job:worker/replica:0/task:0/device:GPU:0']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:worker/task:0/device:GPU:0',)
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['ferrari:60066', 'volvo:60067']}, task_type = 'worker', task_id = 0, num_workers = 2, local_devices = ('/job:worker/task:0/device:GPU:0',), communication = CollectiveCommunication.AUTO


  0%|          | 0/100 [00:00<?, ?it/s]

Number of train images found: 52597
Here
INFO:tensorflow:Collective batch_all_reduce: 2 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 12 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 78 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 38 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 2 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 12 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:Collective batch_all_reduce: 78 all-reduces, num_devices = 1, group_size = 2, communication_hint = AUTO, num_packs = 1
INFO:tensorflow:

100%|██████████| 1/1 [00:00<00:00,  1.29it/s]
